# COVID-19 Statistics for Indiana

## Imports

In [ ]:
import pandas as pd

## Constants

In [ ]:
indianaCaseDataUrl = "https://hub.mph.in.gov/dataset/bd08cdd3-9ab1-4d70-b933-41f9ef7b809d/resource/afaa225d-ac4e-4e80-9190-f6800c366b58/download/covid_report_county_date.xlsx"
indianaCaseDataDictUrl= "https://hub.mph.in.gov/dataset/bd08cdd3-9ab1-4d70-b933-41f9ef7b809d/resource/5ff3931f-aa68-4ee6-ac1d-d6c5d6cca50a/download/covid_report_county_date_dictionary.xlsx"

## Data Import

In [ ]:
df = pd.read_excel(indianaCaseDataUrl)
dfDict = pd.read_excel(indianaCaseDataDictUrl)

## Data Assessment

In [ ]:
df

### Missing Data

In [ ]:
numberOfRows = df.shape[0]
100 * (numberOfRows - df.count()) / numberOfRows

No missing data means we don't need to drop data or impute values.

### Dataframe column data types

In [ ]:
df.dtypes

#### Fix incorrect column data types

In [ ]:
df.DATE = pd.to_datetime(df.DATE)
df.COUNTY_NAME = df.COUNTY_NAME.astype('category')

### Updated column data types

In [ ]:
df.dtypes

In [ ]:
df.COVID_COUNT.sum()